# [PySpark Tutorial](https://sparkbyexamples.com/pyspark-tutorial/)

*   Spark is useful for applications that require a highly distributed persistent, and pipelined processing.
*   Start a project in Pandas with a limited sample (**less than 1 millions rows and 1000 columns**) to explore and migrate to Spark.
*   Spark is useful for **Natural Language Processing and Computer Vision** applications, which typically require alot of calculations.

## Libraries


In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.ml.feature import *
from pyspark.ml.regression import *

import pandas as pd
import numpy as np
from sklearn import datasets

## Functions

In [ ]:
# adjust data types
def convertColumn(df, columns, newType):
  for col in columns: 
     df = df.withColumn(col, df[col].cast(newType))
  return df

## Part 1: Creating Dataframe
- Retrieving Data
- Visualizing Dataframe

### Retrieving Data 

In [ ]:
# start a spark session
spark = SparkSession.builder.appName("Practice").getOrCreate()

# checking spark session
spark

In [ ]:
# store housing dataset
housing_df = datasets.fetch_california_housing()

# show housing dataset info
[print(k, v, '\n') for k,v in housing_df.items()]

In [ ]:
# store dataset in pandas dataframe
df = pd.DataFrame(data=housing_df.data, columns=housing_df.feature_names)
df['y'] = housing_df.target

# convert between spark_df & pandas_df
spark_df = spark.createDataFrame(df.copy()) 
pandas_df = spark_df.toPandas()

# remove 1st row of dummy headers for CSV files
#spark_df  = spark.read.option('header','true').csv('file_path')

### Dataframe Info & [Data Types](https://sparkbyexamples.com/spark/spark-sql-dataframe-data-types/)

In [ ]:
# stats between spark_df & pandas_df
spark_df.describe().show()
pandas_df.describe()

In [ ]:
# make copy of spark_df 
df = spark_df.alias('df')

# check if copy & orignal are same object
id(df) == id(spark_df)

In [ ]:
# identify data types
df.printSchema()
df.dtypes

In [ ]:
# change dtype of certain columns
convertColumn(df, df.columns,FloatType()).printSchema()

### Visualizing Dataframe

In [ ]:
# look at first few rows
df.head(2)

In [ ]:
# # look at first few rows another way
df.show(n=2,truncate=10, vertical=True)

In [ ]:
# show columns
print(df.columns[:], '\n')

# ref column by name
print(df['MedInc'], '\n')

In [ ]:
# selecting data & target columns
df.select(df.columns[:-1]).show()
df.select(df.columns[-1:]).show()

## Part 2: [Updating Dataframe](https://sparkbyexamples.com/pyspark/pyspark-update-a-column-with-value/)
- Manipulating Dataframe
- Dropping / Replacing Data
- Imputing Missing Values

### Manipulating Dataframe

In [ ]:
# renaming columns
df.withColumnRenamed('HouseAge','HouseLife').show()

In [ ]:
# adding new columns based on existing columns
df.withColumn('TripleSpace', df['AveBedrms']*3).show()

In [ ]:
# add new column with constants
df.withColumn("Rewards", F.lit(None)).show()
df.withColumn("Constant", F.lit(1.0)).show()
df.withColumn("Constant", F.lit('fill')).show()

+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+-------+
|MedInc|HouseAge|          AveRooms|         AveBedrms|Population|          AveOccup|Latitude|Longitude|    y|Rewards|
+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+-------+
|8.3252|    41.0| 6.984126984126984|1.0238095238095237|     322.0|2.5555555555555554|   37.88|  -122.23|4.526|   null|
|8.3014|    21.0| 6.238137082601054|0.9718804920913884|    2401.0| 2.109841827768014|   37.86|  -122.22|3.585|   null|
|7.2574|    52.0| 8.288135593220339| 1.073446327683616|     496.0|2.8022598870056497|   37.85|  -122.24|3.521|   null|
|5.6431|    52.0|5.8173515981735155|1.0730593607305936|     558.0| 2.547945205479452|   37.85|  -122.25|3.413|   null|
|3.8462|    52.0| 6.281853281853282|1.0810810810810811|     565.0|2.1814671814671813|   37.85|  -122.25|3.422|   null|
|4.0368|    52.0| 4.761658031088083|1.1036269430

In [ ]:
# removing columns
df.drop('HouseAge','Population').show()

+------+------------------+------------------+------------------+--------+---------+-----+
|MedInc|          AveRooms|         AveBedrms|          AveOccup|Latitude|Longitude|    y|
+------+------------------+------------------+------------------+--------+---------+-----+
|8.3252| 6.984126984126984|1.0238095238095237|2.5555555555555554|   37.88|  -122.23|4.526|
|8.3014| 6.238137082601054|0.9718804920913884| 2.109841827768014|   37.86|  -122.22|3.585|
|7.2574| 8.288135593220339| 1.073446327683616|2.8022598870056497|   37.85|  -122.24|3.521|
|5.6431|5.8173515981735155|1.0730593607305936| 2.547945205479452|   37.85|  -122.25|3.413|
|3.8462| 6.281853281853282|1.0810810810810811|2.1814671814671813|   37.85|  -122.25|3.422|
|4.0368| 4.761658031088083|1.1036269430051813| 2.139896373056995|   37.85|  -122.25|2.697|
|3.6591|4.9319066147859925|0.9513618677042801|2.1284046692607004|   37.84|  -122.25|2.992|
|  3.12| 4.797527047913447| 1.061823802163833|1.7882534775888717|   37.84|  -122.25|2.414|

### Encoding & [Missing Points](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Imputer.html)

In [ ]:
# replace values
df.replace(41.0, None).show()

+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+
|MedInc|HouseAge|          AveRooms|         AveBedrms|Population|          AveOccup|Latitude|Longitude|    y|
+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+
|8.3252|    null| 6.984126984126984|1.0238095238095237|     322.0|2.5555555555555554|   37.88|  -122.23|4.526|
|8.3014|    21.0| 6.238137082601054|0.9718804920913884|    2401.0| 2.109841827768014|   37.86|  -122.22|3.585|
|7.2574|    52.0| 8.288135593220339| 1.073446327683616|     496.0|2.8022598870056497|   37.85|  -122.24|3.521|
|5.6431|    52.0|5.8173515981735155|1.0730593607305936|     558.0| 2.547945205479452|   37.85|  -122.25|3.413|
|3.8462|    52.0| 6.281853281853282|1.0810810810810811|     565.0|2.1814671814671813|   37.85|  -122.25|3.422|
|4.0368|    52.0| 4.761658031088083|1.1036269430051813|     413.0| 2.139896373056995|   37.85|  -122.25|2.697|
|

In [ ]:
# replace values in certain column
df.replace(41.0, None, 'HouseAge').show()

+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+
|MedInc|HouseAge|          AveRooms|         AveBedrms|Population|          AveOccup|Latitude|Longitude|    y|
+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+
|8.3252|    null| 6.984126984126984|1.0238095238095237|     322.0|2.5555555555555554|   37.88|  -122.23|4.526|
|8.3014|    21.0| 6.238137082601054|0.9718804920913884|    2401.0| 2.109841827768014|   37.86|  -122.22|3.585|
|7.2574|    52.0| 8.288135593220339| 1.073446327683616|     496.0|2.8022598870056497|   37.85|  -122.24|3.521|
|5.6431|    52.0|5.8173515981735155|1.0730593607305936|     558.0| 2.547945205479452|   37.85|  -122.25|3.413|
|3.8462|    52.0| 6.281853281853282|1.0810810810810811|     565.0|2.1814671814671813|   37.85|  -122.25|3.422|
|4.0368|    52.0| 4.761658031088083|1.1036269430051813|     413.0| 2.139896373056995|   37.85|  -122.25|2.697|
|

In [ ]:
# encoding points
df.replace(to_replace=[37.88, -122.23], value=[1.0, 1.0]).show()

# encoding points in certain column
df.replace(to_replace=[21.0, 50.0], value=[1.0, 1.0], subset='HouseAge').show()

+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+
|MedInc|HouseAge|          AveRooms|         AveBedrms|Population|          AveOccup|Latitude|Longitude|    y|
+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+
|8.3252|    41.0| 6.984126984126984|1.0238095238095237|     322.0|2.5555555555555554|     1.0|      1.0|4.526|
|8.3014|    21.0| 6.238137082601054|0.9718804920913884|    2401.0| 2.109841827768014|   37.86|  -122.22|3.585|
|7.2574|    52.0| 8.288135593220339| 1.073446327683616|     496.0|2.8022598870056497|   37.85|  -122.24|3.521|
|5.6431|    52.0|5.8173515981735155|1.0730593607305936|     558.0| 2.547945205479452|   37.85|  -122.25|3.413|
|3.8462|    52.0| 6.281853281853282|1.0810810810810811|     565.0|2.1814671814671813|   37.85|  -122.25|3.422|
|4.0368|    52.0| 4.761658031088083|1.1036269430051813|     413.0| 2.139896373056995|   37.85|  -122.25|2.697|
|

In [ ]:
# creat new column with missing values replaced
ex = df.replace(41.0, None, 'HouseAge')
ex.withColumn("HouseMissing", F.when(F.col("HouseAge").isNull(), 0).otherwise(F.col("HouseAge"))).show()

+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+------------+
|MedInc|HouseAge|          AveRooms|         AveBedrms|Population|          AveOccup|Latitude|Longitude|    y|HouseMissing|
+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+------------+
|8.3252|    null| 6.984126984126984|1.0238095238095237|     322.0|2.5555555555555554|   37.88|  -122.23|4.526|         0.0|
|8.3014|    21.0| 6.238137082601054|0.9718804920913884|    2401.0| 2.109841827768014|   37.86|  -122.22|3.585|        21.0|
|7.2574|    52.0| 8.288135593220339| 1.073446327683616|     496.0|2.8022598870056497|   37.85|  -122.24|3.521|        52.0|
|5.6431|    52.0|5.8173515981735155|1.0730593607305936|     558.0| 2.547945205479452|   37.85|  -122.25|3.413|        52.0|
|3.8462|    52.0| 6.281853281853282|1.0810810810810811|     565.0|2.1814671814671813|   37.85|  -122.25|3.422|        52.0|
|4.0368|

In [ ]:
# create classifier
imputer = Imputer(strategy='median', 
                  inputCols=['HouseAge'], 
                  outputCols=['out_HouseAge'])

# apply classifier
imputer.fit(ex).transform(ex).show()

+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+------------+
|MedInc|HouseAge|          AveRooms|         AveBedrms|Population|          AveOccup|Latitude|Longitude|    y|out_HouseAge|
+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+------------+
|8.3252|    null| 6.984126984126984|1.0238095238095237|     322.0|2.5555555555555554|   37.88|  -122.23|4.526|        28.0|
|8.3014|    21.0| 6.238137082601054|0.9718804920913884|    2401.0| 2.109841827768014|   37.86|  -122.22|3.585|        21.0|
|7.2574|    52.0| 8.288135593220339| 1.073446327683616|     496.0|2.8022598870056497|   37.85|  -122.24|3.521|        52.0|
|5.6431|    52.0|5.8173515981735155|1.0730593607305936|     558.0| 2.547945205479452|   37.85|  -122.25|3.413|        52.0|
|3.8462|    52.0| 6.281853281853282|1.0810810810810811|     565.0|2.1814671814671813|   37.85|  -122.25|3.422|        52.0|
|4.0368|

In [ ]:
print(f"Strategy: {imputer.getStrategy()}")
print(f"Error: {imputer.getRelativeError()}")

Strategy: median
Error: 0.001


## Part 3: Filtering & Sorting
* [Filtering Dataframe](https://sparkbyexamples.com/pyspark/pyspark-where-filter/)
* Filtering by conditions & arrays

### Filtering Dataframe

In [ ]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+
|summary|            MedInc|          HouseAge|          AveRooms|         AveBedrms|        Population|          AveOccup|          Latitude|          Longitude|                 y|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+
|  count|             20640|             20640|             20640|             20640|             20640|             20640|             20640|              20640|             20640|
|   mean| 3.870671002906976|28.639486434108527| 5.428999742190385| 1.096675149606209|1425.4767441860465|3.0706551594363716|  35.6318614341086|-119.56970445736394|2.0685581690891293|
| stddev|1.8998217179452666|12.585557612111632|2.4741731394243245|0.4739108567954673|1132.

In [ ]:
# filter column with condition
df.filter(df['AveRooms'] > 3).show()

# filter column with oposite condition
df.filter(~(df['AveRooms'] > 3)).show()

# filter column with multiple conditions
df.filter( (df['HouseAge'] > 51.0) & (df['Latitude'] == 37.88) ).show()

# filter based on values
values = [1.0, 52.0]
df.filter(df['HouseAge'].isin(values)).show()

# filter column using SQL col() function
df.filter(F.col('HouseAge') == 1.0).show()

+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+
|MedInc|HouseAge|          AveRooms|         AveBedrms|Population|          AveOccup|Latitude|Longitude|    y|
+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+
|8.3252|    41.0| 6.984126984126984|1.0238095238095237|     322.0|2.5555555555555554|   37.88|  -122.23|4.526|
|8.3014|    21.0| 6.238137082601054|0.9718804920913884|    2401.0| 2.109841827768014|   37.86|  -122.22|3.585|
|7.2574|    52.0| 8.288135593220339| 1.073446327683616|     496.0|2.8022598870056497|   37.85|  -122.24|3.521|
|5.6431|    52.0|5.8173515981735155|1.0730593607305936|     558.0| 2.547945205479452|   37.85|  -122.25|3.413|
|3.8462|    52.0| 6.281853281853282|1.0810810810810811|     565.0|2.1814671814671813|   37.85|  -122.25|3.422|
|4.0368|    52.0| 4.761658031088083|1.1036269430051813|     413.0| 2.139896373056995|   37.85|  -122.25|2.697|
|

### Odering & Sorting
More details can be found [here](https://sparkbyexamples.com/pyspark/pyspark-orderby-and-sort-explained/)

In [ ]:
# sorting using the sort() function
df.sort(df['y'], df['Population']).show()

# sort by descending & ascending 
df.sort(df['y'].asc(), df['Population'].desc()).show()

+------+--------+------------------+------------------+----------+------------------+--------+---------+-------+
|MedInc|HouseAge|          AveRooms|         AveBedrms|Population|          AveOccup|Latitude|Longitude|      y|
+------+--------+------------------+------------------+----------+------------------+--------+---------+-------+
| 0.536|    36.0|             12.25|               3.5|      18.0|              2.25|   40.31|  -123.17|0.14999|
|1.6607|    16.0|6.7105263157894735|1.9210526315789473|      85.0| 2.236842105263158|   39.71|  -122.74|0.14999|
|   2.1|    19.0| 3.774390243902439|1.4573170731707317|     490.0|2.9878048780487805|    36.4|  -117.02|0.14999|
|4.1932|    52.0| 3.568888888888889|1.1866666666666668|     628.0|2.7911111111111113|   34.24|  -117.86|0.14999|
|2.3667|    39.0| 3.572463768115942|1.2173913043478262|     259.0|1.8768115942028984|   34.15|  -118.33|  0.175|
|0.7917|    52.0| 2.018867924528302| 1.490566037735849|     167.0| 3.150943396226415|   37.95|  

In [ ]:
# sorting using groupBy and aggregate
df.groupBy('HouseAge').sum().show()

df.groupBy('HouseAge').avg().show()

df.groupBy('HouseAge').mean().show()

df.groupBy('HouseAge').count().show()

+--------+------------------+-------------+------------------+------------------+---------------+------------------+------------------+-------------------+------------------+
|HouseAge|       sum(MedInc)|sum(HouseAge)|     sum(AveRooms)|    sum(AveBedrms)|sum(Population)|     sum(AveOccup)|     sum(Latitude)|     sum(Longitude)|            sum(y)|
+--------+------------------+-------------+------------------+------------------+---------------+------------------+------------------+-------------------+------------------+
|     8.0| 918.6237000000001|       1648.0|1293.7791726272803|242.07050979855805|       415781.0| 692.3128971076675| 7363.229999999998|-24579.970000000005|400.49403999999987|
|     7.0|          781.2799|       1225.0|1091.4008105186742| 200.3957193923206|       457289.0| 513.9658917400818| 6220.250000000001| -20844.51000000001| 338.2680599999999|
|    49.0| 475.7662999999999|       6566.0| 665.5445193661704|139.76969351787426|       132552.0| 364.6753501035802| 4826.019

## Part 4: Machine Learning

### Linear Regression

In [ ]:
# check dtype
df.printSchema()
df.columns[:-1]

root
 |-- MedInc: double (nullable = true)
 |-- HouseAge: double (nullable = true)
 |-- AveRooms: double (nullable = true)
 |-- AveBedrms: double (nullable = true)
 |-- Population: double (nullable = true)
 |-- AveOccup: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- y: double (nullable = true)



['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [ ]:
# crunch predictors into 1D array
predictors = VectorAssembler(inputCols=df.columns[:-1], 
                             outputCol="Independent Features")
output = predictors.transform(df)

# show new df & new column
predictors.transform(df).show()
print(predictors.transform(df).columns[-1:])

# finialized dataframe for machine learning
predictors.transform(df).select("Independent Features","y").show()
finalized_data = predictors.transform(df).select("Independent Features","y")

+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+--------------------+
|MedInc|HouseAge|          AveRooms|         AveBedrms|Population|          AveOccup|Latitude|Longitude|    y|Independent Features|
+------+--------+------------------+------------------+----------+------------------+--------+---------+-----+--------------------+
|8.3252|    41.0| 6.984126984126984|1.0238095238095237|     322.0|2.5555555555555554|   37.88|  -122.23|4.526|[8.3252,41.0,6.98...|
|8.3014|    21.0| 6.238137082601054|0.9718804920913884|    2401.0| 2.109841827768014|   37.86|  -122.22|3.585|[8.3014,21.0,6.23...|
|7.2574|    52.0| 8.288135593220339| 1.073446327683616|     496.0|2.8022598870056497|   37.85|  -122.24|3.521|[7.2574,52.0,8.28...|
|5.6431|    52.0|5.8173515981735155|1.0730593607305936|     558.0| 2.547945205479452|   37.85|  -122.25|3.413|[5.6431,52.0,5.81...|
|3.8462|    52.0| 6.281853281853282|1.0810810810810811|     565.0|2.18146718

In [ ]:
# train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25])

# select model 
regressor = LinearRegression(featuresCol='Independent Features', 
                             labelCol='y', 
                             predictionCol='y_pred')
# apply model
regressor = regressor.fit(train_data)

In [ ]:
# prediction
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+-------+--------------------+
|Independent Features|      y|              y_pred|
+--------------------+-------+--------------------+
|[0.4999,28.0,7.67...|5.00001|  0.8512224678613407|
|[0.4999,52.0,2.6,...|  0.906|  1.0218125095476367|
|[0.536,46.0,3.142...|  0.875|  0.9938453979739492|
|[0.536,46.0,9.0,1...|   3.75|  0.7422051298725378|
|[0.716,39.0,4.730...|  1.042|  1.0843551357255805|
|[0.7403,37.0,4.49...|  0.686|  1.1576735241587457|
|[0.75,52.0,2.8235...|  1.625|   1.351316834640663|
|[0.7591,16.0,3.42...|    3.5|   1.035451917576303|
|[0.8024,48.0,5.13...|  0.583| 0.27421229004073666|
|[0.8026,23.0,5.36...|  1.125|  0.9714549911745252|
|[0.8054,52.0,2.90...|    0.6|  1.1416936639779962|
|[0.8172,52.0,6.10...|  0.853|   1.331327497291781|
|[0.8639,28.0,4.28...|  0.494|  0.7079751580467359|
|[0.8641,37.0,4.07...|  0.842|  1.2319380657896488|
|[0.8804,36.0,2.71...|    4.5|  1.2498964710823444|
|[0.8907,34.0,2.28...|  1.542|  0.9643737324993964|
|[0.8941,33.

In [ ]:
# regression results
print(f"regressionCoeffs: {regressor.coefficients}")
print(f"meanAbsoluteError: {round(pred_results.meanAbsoluteError, 3)}")
print(f"meanSquaredError: {round(pred_results.meanSquaredError, 3)}")

regressionCoeffs: [0.4442807480586436,0.009633282335989502,-0.12269262909671293,0.7671444535203975,-3.6269188051184176e-06,-0.0034029775477794296,-0.41784021629767765,-0.43497979732253106]
meanAbsoluteError: 0.528
meanSquaredError: 0.533
